# Setting Up the Environment

In [1]:
import os
import json

In [2]:
import cv2
import keras

Using TensorFlow backend.


In [3]:
from keras import backend as K
from keras.models import Model
from keras.layers import Input
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.losses import binary_crossentropy
from keras.callbacks import Callback, ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [4]:
from tqdm import tqdm

In [5]:
from sklearn.model_selection import train_test_split

# Preprocessing the Input File

In [6]:
train_df = pd.read_csv('../../3-1/AppofAI/train.csv')

In [7]:
train_df['ImageId'] = train_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
train_df['ClassId'] = train_df['ImageId_ClassId'].apply(lambda x: x.split('_')[1])
train_df['HasMask'] = ~ train_df['EncodedPixels'].isna()

In [8]:
print(train_df.shape)
train_df.head()

(50272, 5)


,ImageId_ClassId,EncodedPixels,ImageId,ClassId,HasMask
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...,0002cc93b.jpg,1,True
1,0002cc93b.jpg_2,NaN,0002cc93b.jpg,2,False
2,0002cc93b.jpg_3,NaN,0002cc93b.jpg,3,False
3,0002cc93b.jpg_4,NaN,0002cc93b.jpg,4,False
4,00031f466.jpg_1,NaN,00031f466.jpg,1,False


In [9]:
mask_count_df = train_df.groupby('ImageId').agg(np.sum).reset_index()
mask_count_df.sort_values('HasMask', ascending=False, inplace=True)
print(mask_count_df.shape)
mask_count_df.head()

(12568, 2)


,ImageId,HasMask
10803,db4867ee8.jpg,3.0
11776,ef24da2ba.jpg,3.0
6284,7f30b9c64.jpg,2.0
9421,bf0c81db6.jpg,2.0
9615,c314f43f3.jpg,2.0


In [10]:
sub_df = pd.read_csv('../../3-1/AppofAI/sample_submission.csv')
sub_df['ImageId'] = sub_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
test_imgs = pd.DataFrame(sub_df['ImageId'].unique(), columns=['ImageId'])

# Mask Encoding and Decoding

In [11]:
def mask2rle(img):
    
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle2mask(mask_rle, shape=(256,1600)):
    
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

def build_masks(rles, input_shape):
    depth = len(rles)
    height, width = input_shape
    masks = np.zeros((height, width, depth))
    
    for i, rle in enumerate(rles):
        if type(rle) is str:
            masks[:, :, i] = rle2mask(rle, (width, height))
    
    return masks

def build_rles(masks):
    width, height, depth = masks.shape
    
    rles = [mask2rle(masks[:, :, i])
            for i in range(depth)]
    
    return rles

# Loss function - Balanced Cross Entropy + Dice Loss

In [12]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

# Data generator - To load images in batches 

In [13]:
class DataGenerator(keras.utils.Sequence):
    
    def __init__(self, list_IDs, df, target_df=None, mode='fit',
                 base_path='../../3-1/AppofAI/train_images',
                 batch_size=32, dim=(256, 1600), n_channels=1,
                 n_classes=4, random_state=2019, shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.df = df
        self.mode = mode
        self.base_path = base_path
        self.target_df = target_df
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.random_state = random_state
        
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.list_IDs) / self.batch_size))
    
    def __getitem__(self, index):

        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        list_IDs_batch = [self.list_IDs[k] for k in indexes]
        
        X = self.__generate_X(list_IDs_batch)
        
        if self.mode == 'fit':
            y = self.__generate_y(list_IDs_batch)
            return X, y
        
        elif self.mode == 'predict':
            return X

        else:
            raise AttributeError('The mode parameter should be set to "fit" or "predict".')
            
    def on_epoch_end(self):
        
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.seed(self.random_state)
            np.random.shuffle(self.indexes)
    
    def __generate_X(self, list_IDs_batch):

        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        
        for i, ID in enumerate(list_IDs_batch):
            im_name = self.df['ImageId'].iloc[ID]
            img_path = f"{self.base_path}/{im_name}"
            img = self.__load_grayscale(img_path)
            
            X[i,] = img
            
        return X
    
    def __generate_y(self, list_IDs_batch):
        y = np.empty((self.batch_size, *self.dim, self.n_classes), dtype=int)
        
        for i, ID in enumerate(list_IDs_batch):
            im_name = self.df['ImageId'].iloc[ID]
            image_df = self.target_df[self.target_df['ImageId'] == im_name]
            
            rles = image_df['EncodedPixels'].values
            masks = build_masks(rles, input_shape=self.dim)
            
            y[i, ] = masks

        return y
    
    def __load_grayscale(self, img_path):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = img.astype(np.float32) / 255.
        img = np.expand_dims(img, axis=-1)

        return img
    
    def __load_rgb(self, img_path):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32) / 255.

        return img

In [14]:
BATCH_SIZE = 10

train_idx, val_idx = train_test_split(
    mask_count_df.index, random_state=2019, test_size=0.15
)

train_generator = DataGenerator(
    train_idx, 
    df=mask_count_df,
    target_df=train_df,
    batch_size=BATCH_SIZE, 
    n_classes=4
)

val_generator = DataGenerator(
    val_idx, 
    df=mask_count_df,
    target_df=train_df,
    batch_size=BATCH_SIZE, 
    n_classes=4
)


# Model Architecture

In [15]:
def build_model(input_shape):
    inputs = Input(input_shape)

    c1 = Conv2D(8, (3, 3), activation='elu', padding='same') (inputs)
    c1 = Conv2D(8, (3, 3), activation='elu', padding='same') (c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = Conv2D(16, (3, 3), activation='elu', padding='same') (p1)
    c2 = Conv2D(16, (3, 3), activation='elu', padding='same') (c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(32, (3, 3), activation='elu', padding='same') (p2)
    c3 = Conv2D(32, (3, 3), activation='elu', padding='same') (c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = Conv2D(64, (3, 3), activation='elu', padding='same') (p3)
    c4 = Conv2D(64, (3, 3), activation='elu', padding='same') (c4)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

    c5 = Conv2D(64, (3, 3), activation='elu', padding='same') (p4)
    c5 = Conv2D(64, (3, 3), activation='elu', padding='same') (c5)
    p5 = MaxPooling2D(pool_size=(2, 2)) (c5)

    c55 = Conv2D(128, (3, 3), activation='elu', padding='same') (p5)
    c55 = Conv2D(128, (3, 3), activation='elu', padding='same') (c55)
    
    u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c55)
    u6 = concatenate([u6, c5])
    c6 = Conv2D(64, (3, 3), activation='elu', padding='same') (u6)
    c6 = Conv2D(64, (3, 3), activation='elu', padding='same') (c6)

    u71 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
    u71 = concatenate([u71, c4])
    c71 = Conv2D(32, (3, 3), activation='elu', padding='same') (u71)
    c61 = Conv2D(32, (3, 3), activation='elu', padding='same') (c71)

    u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c61)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(32, (3, 3), activation='elu', padding='same') (u7)
    c7 = Conv2D(32, (3, 3), activation='elu', padding='same') (c7)

    u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(16, (3, 3), activation='elu', padding='same') (u8)
    c8 = Conv2D(16, (3, 3), activation='elu', padding='same') (c8)

    u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(8, (3, 3), activation='elu', padding='same') (u9)
    c9 = Conv2D(8, (3, 3), activation='elu', padding='same') (c9)

    outputs = Conv2D(4, (1, 1), activation='sigmoid') (c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss=bce_dice_loss, metrics=[dice_coef])
    
    return model

# Training the model

In [ ]:
model = build_model((256, 1600, 1))
model.summary()


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 1600, 1) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 1600, 8) 80          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 1600, 8) 584         conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 128, 800, 8)  0           conv2d_2[0][0]                 

concatenate_5 (Concatenate)     (None, 256, 1600, 16 0           conv2d_transpose_5[0][0]         
                                                                 conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_21 (Conv2D)              (None, 256, 1600, 8) 1160        concatenate_5[0][0]              
__________________________________________________________________________________________________
conv2d_22 (Conv2D)              (None, 256, 1600, 8) 584         conv2d_21[0][0]                  
__________________________________________________________________________________________________
conv2d_23 (Conv2D)              (None, 256, 1600, 4) 36          conv2d_22[0][0]                  
Total params: 600,612
Trainable params: 600,612
Non-trainable params: 0
__________________________________________________________________________________________________


In [ ]:
checkpoint = ModelCheckpoint(
    'model.h5', 
    monitor='val_dice_coef', 
    verbose=0, 
    save_best_only=True, 
    save_weights_only=False,
    mode='auto'
)

history = model.fit_generator(
    train_generator,
    validation_data=val_generator,
    callbacks=[checkpoint],
    use_multiprocessing=False,
    workers=1,
    epochs=7
)


Epoch 1/7


 170/1068 [===>..........................] - ETA: 5:22:09 - loss: 1.7052 - dice_coef: 0.03 - ETA: 4:08:59 - loss: 1.6828 - dice_coef: 0.03 - ETA: 3:44:12 - loss: 1.6646 - dice_coef: 0.02 - ETA: 3:33:18 - loss: 1.6460 - dice_coef: 0.02 - ETA: 3:28:31 - loss: 1.6298 - dice_coef: 0.02 - ETA: 3:22:20 - loss: 1.6059 - dice_coef: 0.02 - ETA: 3:19:52 - loss: 1.5869 - dice_coef: 0.02 - ETA: 3:17:01 - loss: 1.5669 - dice_coef: 0.01 - ETA: 3:14:50 - loss: 1.5426 - dice_coef: 0.01 - ETA: 3:14:55 - loss: 1.5189 - dice_coef: 0.01 - ETA: 3:14:54 - loss: 1.4930 - dice_coef: 0.01 - ETA: 3:12:58 - loss: 1.4674 - dice_coef: 0.01 - ETA: 3:11:08 - loss: 1.4406 - dice_coef: 0.01 - ETA: 3:09:53 - loss: 1.4144 - dice_coef: 0.01 - ETA: 3:09:50 - loss: 1.3916 - dice_coef: 0.01 - ETA: 3:10:20 - loss: 1.3707 - dice_coef: 0.01 - ETA: 3:09:49 - loss: 1.3512 - dice_coef: 0.01 - ETA: 3:09:21 - loss: 1.3348 - dice_coef: 0.01 - ETA: 3:09:46 - loss: 1.3187 - dice_coef: 0.01 - ETA: 3:08:45 - loss: 1.3041 - dice_coef: 0.

 340/1068 [========>.....................] - ETA: 2:34:41 - loss: 1.0588 - dice_coef: 0.02 - ETA: 2:34:30 - loss: 1.0587 - dice_coef: 0.02 - ETA: 2:34:20 - loss: 1.0585 - dice_coef: 0.02 - ETA: 2:34:09 - loss: 1.0582 - dice_coef: 0.02 - ETA: 2:33:59 - loss: 1.0579 - dice_coef: 0.02 - ETA: 2:33:48 - loss: 1.0578 - dice_coef: 0.02 - ETA: 2:33:38 - loss: 1.0575 - dice_coef: 0.02 - ETA: 2:33:27 - loss: 1.0577 - dice_coef: 0.02 - ETA: 2:33:16 - loss: 1.0574 - dice_coef: 0.02 - ETA: 2:33:06 - loss: 1.0573 - dice_coef: 0.02 - ETA: 2:32:55 - loss: 1.0572 - dice_coef: 0.02 - ETA: 2:32:44 - loss: 1.0570 - dice_coef: 0.02 - ETA: 2:32:33 - loss: 1.0567 - dice_coef: 0.02 - ETA: 2:32:23 - loss: 1.0566 - dice_coef: 0.02 - ETA: 2:32:14 - loss: 1.0564 - dice_coef: 0.02 - ETA: 2:32:03 - loss: 1.0560 - dice_coef: 0.02 - ETA: 2:31:52 - loss: 1.0557 - dice_coef: 0.02 - ETA: 2:31:40 - loss: 1.0555 - dice_coef: 0.02 - ETA: 2:31:29 - loss: 1.0555 - dice_coef: 0.02 - ETA: 2:31:18 - loss: 1.0553 - dice_coef: 0.

 510/1068 [=============>................] - ETA: 2:04:37 - loss: 1.0378 - dice_coef: 0.02 - ETA: 2:04:26 - loss: 1.0378 - dice_coef: 0.02 - ETA: 2:04:16 - loss: 1.0377 - dice_coef: 0.02 - ETA: 2:04:06 - loss: 1.0376 - dice_coef: 0.02 - ETA: 2:03:55 - loss: 1.0375 - dice_coef: 0.02 - ETA: 2:03:45 - loss: 1.0374 - dice_coef: 0.02 - ETA: 2:03:34 - loss: 1.0375 - dice_coef: 0.02 - ETA: 2:03:24 - loss: 1.0374 - dice_coef: 0.02 - ETA: 2:03:14 - loss: 1.0373 - dice_coef: 0.02 - ETA: 2:03:03 - loss: 1.0373 - dice_coef: 0.02 - ETA: 2:02:52 - loss: 1.0373 - dice_coef: 0.02 - ETA: 2:02:40 - loss: 1.0372 - dice_coef: 0.02 - ETA: 2:02:30 - loss: 1.0372 - dice_coef: 0.02 - ETA: 2:02:19 - loss: 1.0371 - dice_coef: 0.02 - ETA: 2:02:09 - loss: 1.0369 - dice_coef: 0.02 - ETA: 2:01:59 - loss: 1.0369 - dice_coef: 0.02 - ETA: 2:01:49 - loss: 1.0369 - dice_coef: 0.02 - ETA: 2:01:38 - loss: 1.0368 - dice_coef: 0.02 - ETA: 2:01:27 - loss: 1.0367 - dice_coef: 0.02 - ETA: 2:01:17 - loss: 1.0366 - dice_coef: 0.

 680/1068 [==================>...........] - ETA: 1:34:42 - loss: 1.0297 - dice_coef: 0.03 - ETA: 1:34:32 - loss: 1.0297 - dice_coef: 0.03 - ETA: 1:34:21 - loss: 1.0296 - dice_coef: 0.03 - ETA: 1:34:10 - loss: 1.0296 - dice_coef: 0.03 - ETA: 1:33:59 - loss: 1.0296 - dice_coef: 0.03 - ETA: 1:33:48 - loss: 1.0295 - dice_coef: 0.03 - ETA: 1:33:38 - loss: 1.0294 - dice_coef: 0.03 - ETA: 1:33:27 - loss: 1.0294 - dice_coef: 0.03 - ETA: 1:33:16 - loss: 1.0294 - dice_coef: 0.03 - ETA: 1:33:05 - loss: 1.0294 - dice_coef: 0.03 - ETA: 1:32:54 - loss: 1.0294 - dice_coef: 0.03 - ETA: 1:32:44 - loss: 1.0293 - dice_coef: 0.03 - ETA: 1:32:33 - loss: 1.0293 - dice_coef: 0.03 - ETA: 1:32:22 - loss: 1.0293 - dice_coef: 0.03 - ETA: 1:32:12 - loss: 1.0292 - dice_coef: 0.03 - ETA: 1:32:01 - loss: 1.0292 - dice_coef: 0.03 - ETA: 1:31:50 - loss: 1.0292 - dice_coef: 0.03 - ETA: 1:31:39 - loss: 1.0291 - dice_coef: 0.03 - ETA: 1:31:29 - loss: 1.0290 - dice_coef: 0.03 - ETA: 1:31:18 - loss: 1.0290 - dice_coef: 0.

 856/1068 [=======================>......] - ETA: 1:05:18 - loss: 1.0244 - dice_coef: 0.03 - ETA: 1:05:08 - loss: 1.0244 - dice_coef: 0.03 - ETA: 1:04:58 - loss: 1.0244 - dice_coef: 0.03 - ETA: 1:04:47 - loss: 1.0244 - dice_coef: 0.03 - ETA: 1:04:37 - loss: 1.0243 - dice_coef: 0.03 - ETA: 1:04:27 - loss: 1.0243 - dice_coef: 0.03 - ETA: 1:04:16 - loss: 1.0243 - dice_coef: 0.03 - ETA: 1:04:06 - loss: 1.0243 - dice_coef: 0.03 - ETA: 1:03:56 - loss: 1.0242 - dice_coef: 0.03 - ETA: 1:03:45 - loss: 1.0242 - dice_coef: 0.03 - ETA: 1:03:35 - loss: 1.0241 - dice_coef: 0.03 - ETA: 1:03:25 - loss: 1.0241 - dice_coef: 0.03 - ETA: 1:03:14 - loss: 1.0241 - dice_coef: 0.03 - ETA: 1:03:04 - loss: 1.0240 - dice_coef: 0.03 - ETA: 1:02:54 - loss: 1.0241 - dice_coef: 0.03 - ETA: 1:02:43 - loss: 1.0241 - dice_coef: 0.03 - ETA: 1:02:33 - loss: 1.0240 - dice_coef: 0.03 - ETA: 1:02:23 - loss: 1.0240 - dice_coef: 0.03 - ETA: 1:02:12 - loss: 1.0240 - dice_coef: 0.03 - ETA: 1:02:02 - loss: 1.0239 - dice_coef: 0.

1034/1068 [============================>.] - ETA: 35:21 - loss: 1.0210 - dice_coef: 0.03 - ETA: 35:11 - loss: 1.0211 - dice_coef: 0.03 - ETA: 35:01 - loss: 1.0210 - dice_coef: 0.03 - ETA: 34:51 - loss: 1.0210 - dice_coef: 0.03 - ETA: 34:41 - loss: 1.0210 - dice_coef: 0.03 - ETA: 34:31 - loss: 1.0210 - dice_coef: 0.03 - ETA: 34:21 - loss: 1.0210 - dice_coef: 0.03 - ETA: 34:11 - loss: 1.0210 - dice_coef: 0.03 - ETA: 34:01 - loss: 1.0209 - dice_coef: 0.03 - ETA: 33:51 - loss: 1.0209 - dice_coef: 0.03 - ETA: 33:41 - loss: 1.0209 - dice_coef: 0.03 - ETA: 33:31 - loss: 1.0209 - dice_coef: 0.03 - ETA: 33:21 - loss: 1.0209 - dice_coef: 0.03 - ETA: 33:11 - loss: 1.0209 - dice_coef: 0.03 - ETA: 33:01 - loss: 1.0209 - dice_coef: 0.03 - ETA: 32:51 - loss: 1.0209 - dice_coef: 0.03 - ETA: 32:41 - loss: 1.0208 - dice_coef: 0.03 - ETA: 32:31 - loss: 1.0208 - dice_coef: 0.03 - ETA: 32:21 - loss: 1.0208 - dice_coef: 0.03 - ETA: 32:11 - loss: 1.0208 - dice_coef: 0.03 - ETA: 32:01 - loss: 1.0208 - dice_co

1068/1068 [==============================] - ETA: 5:31 - loss: 1.0183 - dice_coef: 0.035 - ETA: 5:21 - loss: 1.0183 - dice_coef: 0.035 - ETA: 5:11 - loss: 1.0183 - dice_coef: 0.035 - ETA: 5:01 - loss: 1.0183 - dice_coef: 0.035 - ETA: 4:51 - loss: 1.0183 - dice_coef: 0.035 - ETA: 4:41 - loss: 1.0182 - dice_coef: 0.035 - ETA: 4:30 - loss: 1.0182 - dice_coef: 0.035 - ETA: 4:20 - loss: 1.0181 - dice_coef: 0.035 - ETA: 4:10 - loss: 1.0181 - dice_coef: 0.035 - ETA: 4:00 - loss: 1.0181 - dice_coef: 0.035 - ETA: 3:50 - loss: 1.0180 - dice_coef: 0.035 - ETA: 3:40 - loss: 1.0180 - dice_coef: 0.035 - ETA: 3:30 - loss: 1.0179 - dice_coef: 0.035 - ETA: 3:20 - loss: 1.0179 - dice_coef: 0.035 - ETA: 3:10 - loss: 1.0179 - dice_coef: 0.035 - ETA: 3:00 - loss: 1.0179 - dice_coef: 0.035 - ETA: 2:50 - loss: 1.0178 - dice_coef: 0.035 - ETA: 2:40 - loss: 1.0178 - dice_coef: 0.036 - ETA: 2:30 - loss: 1.0178 - dice_coef: 0.036 - ETA: 2:20 - loss: 1.0178 - dice_coef: 0.036 - ETA: 2:10 - loss: 1.0178 - dice_coe

# Evaluation

In [ ]:
with open('history.json', 'w') as f:
    json.dump(history.history, f)

history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot()

In [ ]:
model.load_weights('model.h5')
test_df = []

for i in range(0, test_imgs.shape[0], 500):
    batch_idx = list(
        range(i, min(test_imgs.shape[0], i + 500))
    )
    
    test_generator = DataGenerator(
        batch_idx,
        df=test_imgs,
        shuffle=False,
        mode='predict',
        base_path='../input/severstal-steel-defect-detection/test_images',
        target_df=sub_df,
        batch_size=1,
        n_classes=4
    )
    
    batch_pred_masks = model.predict_generator(
        test_generator, 
        workers=1,
        verbose=1,
        use_multiprocessing=False
    )
    
    for j, b in tqdm(enumerate(batch_idx)):
        filename = test_imgs['ImageId'].iloc[b]
        image_df = sub_df[sub_df['ImageId'] == filename].copy()
        
        pred_masks = batch_pred_masks[j, ].round().astype(int)
        pred_rles = build_rles(pred_masks)
        
        image_df['EncodedPixels'] = pred_rles
        test_df.append(image_df)

In [ ]:
test_df = pd.concat(test_df)
test_df.drop(columns='ImageId', inplace=True)
test_df.to_csv('test_predictions.csv', index=False)